## **Global Code**

In [3]:
import torch
import keras
import keras.layers as L
import numpy as np
from tqdm.auto import tqdm
import pandas as pd
import json
import os
from transformers import pipeline
from sklearn.utils.class_weight import compute_class_weight

In [4]:
offence_class = {
    0:"No offence", 
    1:"Offence + No card", 
    2:"Offence + Yellow card", 
    3:"Offence + Red card"
}

In [5]:
action_class = {
    0:"Tackling", 
    1:"Standing tackling", 
    2:"High leg", 
    3:"Holding", 
    4:"Pushing",
    5:"Elbowing", 
    6:"Challenge", 
    7:"Dive", 
    8:"Dont know"
}

### Prepare pretrained model pipeline

In [6]:
#Download model
pipe_soccer = pipeline("video-classification", model="anirudhmu/videomae-base-finetuned-soccer-action-recognitionx4")

pipe_soccer.model.classifier = torch.nn.Identity() #removes the final layer
pipe_soccer.postprocess = lambda x, *a, **k: x.logits #get logits

#use the predict method to check the output shape
pipe_soccer.predict('/Users/khuzam/Desktop/fp/SoccerNet/mvfouls/train/action_4/clip_3.mp4').shape #check shape of output

torch.Size([1, 768])

### Extractor for pipeline only

In [ ]:
def extract(video_path):
    action_features=[]

    #for every clip --> get feature vector
    # number_of_clips= len(os.listdir(action_path))
    # for clip in tqdm(range(0, number_of_clips)):
    # video_path= action_path+'/clip_'+str(clip)+'.mp4'
    clip_features= pipe_soccer.predict(video_path)
    action_features.append(clip_features)

    #if action has < 4 clips --> duplicate last clip (features)
    # while len(action_features) > 0 and len(action_features) < 4:
    #     action_features.append(action_features[-1])

    return np.asarray(action_features)

### Label Extraction (soccernet kit)

This section extracts the following:
1. Data labels using SoccerNet's method
2. Features using the pretrained model 

In [6]:
# Function to load the labels from the json file
def label2vectormerge(folder_path, split, num_views):
	path_annotations = os.path.join(folder_path, split)
	path_annotations = os.path.join(path_annotations, "annotations.json") 

	dictionary_action = {"Tackling":0,"Standing tackling":1,"High leg":2,"Holding":3,"Pushing":4,
                        "Elbowing":5, "Challenge":6, "Dive":7, "Dont know":8}

	if os.path.exists(path_annotations):
		with open(path_annotations) as f:
			train_annotations_data = json.load(f)
	else:
		print("PATH DOES NOT EXISTS")
		exit()

	not_taking = []

	num_classes_action = 8
	num_classes_offence_severity = 4

	labels_action = []
	labels_offence_severity= []
	number_of_actions = []

	total_distribution = torch.zeros(num_classes_offence_severity, num_classes_action)
	distribution_action = torch.zeros(1, num_classes_action)
	distribution_offence_severity = torch.zeros(1, num_classes_offence_severity)

	for actions in train_annotations_data['Actions']:
		action_class = train_annotations_data['Actions'][actions]['Action class']
		offence_class = train_annotations_data['Actions'][actions]['Offence']
		severity_class = train_annotations_data['Actions'][actions]['Severity']


		if action_class == '' or action_class == 'Dont know':
			not_taking.append(actions)
			continue

		if (offence_class == '' or offence_class == 'Between') and action_class != 'Dive':
			not_taking.append(actions)
			continue

		if (severity_class == '' or severity_class == '2.0' or severity_class == '4.0') and action_class != 'Dive' and offence_class != 'No offence' and offence_class != 'No Offence':
			not_taking.append(actions)
			continue

		if offence_class == '' or offence_class == 'Between':
			offence_class = 'Offence'

		if severity_class == '' or severity_class == '2.0' or severity_class == '4.0':
			severity_class = '1.0'

		if num_views == 1:
			for i in range(len(train_annotations_data['Actions'][actions]['Clips'])):
				if offence_class == 'No Offence' or offence_class == 'No offence':
					labels_offence_severity.append(torch.zeros(1, num_classes_offence_severity))
					labels_offence_severity[len(labels_offence_severity)-1][0][0] = 1
					distribution_offence_severity[0][0] += 1
					off_index = 0
				elif offence_class == 'Offence' and severity_class == '1.0':
					labels_offence_severity.append(torch.zeros(1, num_classes_offence_severity))
					labels_offence_severity[len(labels_offence_severity)-1][0][1] = 1
					distribution_offence_severity[0][1] += 1
					off_index = 1
				elif offence_class == 'Offence' and severity_class == '3.0':
					labels_offence_severity.append(torch.zeros(1, num_classes_offence_severity))
					labels_offence_severity[len(labels_offence_severity)-1][0][2] = 1
					distribution_offence_severity[0][2] += 1
					off_index = 2
				elif offence_class == 'Offence' and severity_class == '5.0':
					labels_offence_severity.append(torch.zeros(1, num_classes_offence_severity))
					labels_offence_severity[len(labels_offence_severity)-1][0][3] = 1
					distribution_offence_severity[0][3] += 1
					off_index = 3
				else:
					not_taking.append(actions)
					continue
				labels_action.append(torch.zeros(1, num_classes_action))
				labels_action[len(labels_action)-1][0][dictionary_action[action_class]] = 1
				distribution_action[0][dictionary_action[action_class]] += 1
				total_distribution[off_index][dictionary_action[action_class]] += 1
		else:
			if offence_class == 'No Offence' or offence_class == 'No offence':
				labels_offence_severity.append(torch.zeros(1, num_classes_offence_severity))
				labels_offence_severity[len(labels_offence_severity)-1][0][0] = 1
				distribution_offence_severity[0][0] += 1
				index = 0
			elif offence_class == 'Offence' and severity_class == '1.0':
				labels_offence_severity.append(torch.zeros(1, num_classes_offence_severity))
				labels_offence_severity[len(labels_offence_severity)-1][0][1] = 1
				distribution_offence_severity[0][1] += 1
				index = 1
			elif offence_class == 'Offence' and severity_class == '3.0':
				labels_offence_severity.append(torch.zeros(1, num_classes_offence_severity))
				labels_offence_severity[len(labels_offence_severity)-1][0][2] = 1
				distribution_offence_severity[0][2] += 1
				index = 2
			elif offence_class == 'Offence' and severity_class == '5.0':
				labels_offence_severity.append(torch.zeros(1, num_classes_offence_severity))
				labels_offence_severity[len(labels_offence_severity)-1][0][3] = 1
				distribution_offence_severity[0][3] += 1
				index = 3
			else:
				not_taking.append(actions)
				continue

			number_of_actions.append(actions)
			labels_action.append(torch.zeros(1, num_classes_action))
			labels_action[len(labels_action)-1][0][dictionary_action[action_class]] = 1
			distribution_action[0][dictionary_action[action_class]] += 1
			total_distribution[index][dictionary_action[action_class]] += 1

	return labels_offence_severity, labels_action, distribution_offence_severity[0], distribution_action[0], not_taking, number_of_actions

In [1]:
def extract(main_path, actions_ints):
    all_features= []
    order=[]

    for i in tqdm(actions_ints):
        action_features=[]
        action_path= main_path+'/action_'+str(i)

        #for every clip --> get feature vector
        for video in os.listdir(action_path):
            video_path= main_path+'/action_'+str(i)+'/'+video
            clip_features= pipe_soccer.predict(video_path)
            action_features.append(clip_features)

        #if action has < 4 clips --> duplicate last clip (features)
        while len(action_features) > 0 and len(action_features) < 4:
            action_features.append(action_features[-1])

        all_features.append(action_features)
        order.append(i)

    return all_features, order

In [39]:
#load action numbers for the split (traiin, valiid or test)
action_ints= np.load('./labels/test_labels/number_of_actions.npy')

In [40]:
def extract(main_path):
    all_features= []
    order=[]

    for action in tqdm(np.arange(273)):
        action_features=[]
        action_path= main_path+'/action_'+str(action)

        #for every clip --> get feature vector
        for clip in os.listdir(action_path):
            video_path= action_path+'/'+clip
            # clip_features= pipe_soccer.predict(video_path)
            # action_features.append(clip_features)
            action_features.append(video_path)

        #if action has < 4 clips --> duplicate last clip (features)
        while len(action_features) < 4:
            action_features.append(action_features[-1])

        all_features.append(action_features)
        order.append(action)

    return all_features, order

In [ ]:
#extract features for the split
#input: data file path
train_features, order= extract('./SoccerNet/mvfouls/train')

---

### Read/Prepare Data

In [7]:
#training data
train_x= np.load('/Users/khuzam/Desktop/fp/SoccerNet/train_labels/train_x_ordered.npy')
train_y1= pd.read_csv('/Users/khuzam/Desktop/fp/SoccerNet/train_labels/train.csv')['offence_severity']
train_y1 = train_y1.replace(dict(zip(offence_class.values(), offence_class))).values
train_y2= pd.read_csv('/Users/khuzam/Desktop/fp/SoccerNet/train_labels/train.csv')['action_class']
train_y2 = train_y2.replace(dict(zip(action_class.values(), action_class))).values

In [8]:
#validation data
val_x= np.load('/Users/khuzam/Desktop/fp/SoccerNet/valid_labels/val_x_ordered.npy')
val_y1= pd.read_csv('/Users/khuzam/Desktop/fp/SoccerNet/valid_labels/valid.csv')['offence_severity']
val_y1 = val_y1.replace(dict(zip(offence_class.values(), offence_class))).values
val_y2= pd.read_csv('/Users/khuzam/Desktop/fp/SoccerNet/valid_labels/valid.csv')['action_class']
val_y2 = val_y2.replace(dict(zip(action_class.values(), action_class))).values

In [9]:
#test data
test_x= np.load('/Users/khuzam/Desktop/fp/SoccerNet/test_labels/test_x_ordered.npy')
test_y1= pd.read_csv('/Users/khuzam/Desktop/fp/SoccerNet/test_labels/test.csv')['offence_severity']
test_y1 = test_y1.replace(dict(zip(offence_class.values(), offence_class))).values
test_y2= pd.read_csv('/Users/khuzam/Desktop/fp/SoccerNet/test_labels/test.csv')['action_class']
test_y2 = test_y2.replace(dict(zip(action_class.values(), action_class))).values

In [10]:
#challenge data
chall_x= np.load('/Users/khuzam/Desktop/fp/SoccerNet/challenge_labels/challenge_features.npy')

In [11]:
#reshape data
train_x= np.reshape(train_x, (2319, 4, 768))
val_x= np.reshape(val_x, (321, 4, 768))
test_x= np.reshape(test_x, (251, 4, 768))

In [12]:
#Calculate class weights
class_weights = compute_class_weight(class_weight="balanced", classes=np.unique(train_y1), y=train_y1)
class_weights = dict(zip(range(len(class_weights)), class_weights.tolist()))

In [13]:
#Calculate class weights
class_weights2 = compute_class_weight(class_weight="balanced", classes=np.unique(train_y2), y=train_y2)
class_weights2 = dict(zip(range(len(class_weights2)), class_weights2.tolist()))

In [ ]:
# #class weights
# def categorical_to_sparse(one_hot):
#     return np.array(list(map(lambda x: x.argmax(-1).item(), one_hot)), dtype=np.uint8)

### Model using features extracted
1. pipe_Soccer: video -> features (4, 768)
2. features (input)
3. Layers
4. Classifier (output)

## **Classification Using 2 Seperate models**

#### **Y1 classification**

In [446]:
input_shape=train_x.shape[1:]

**BEST**

In [447]:
keras.utils.clear_session()

model_y1= keras.models.Sequential([
    L.Input(input_shape,),
    L.LSTM(units= 512, return_sequences=True),
    L.LSTM(units=256),
    L.BatchNormalization(),
    L.Dropout(0.2),
    L.Dense(128, activation='relu'),
    L.Dense(64, activation='relu'),
    L.Dropout(0.2),
    L.Dense(units=4, activation='softmax')
])

model_y1.summary()

model_y1.compile(
    loss="categorical_crossentropy", 
    optimizer="adam", 
    metrics=[
        "accuracy",
        "f1_score",  
    ],
    weighted_metrics=[
        "accuracy", 
    ]
)

model_y1.fit(
    train_x,
    keras.utils.to_categorical(train_y1),
    epochs=50,
    batch_size=32,
    validation_data=[val_x, keras.utils.to_categorical(val_y1)],
    class_weight= class_weights,
    callbacks= [
        keras.callbacks.ReduceLROnPlateau(
            patience=5,
            factor=0.1,
            monitor='val_accuracy',
            mode='min'
        ),
        keras.callbacks.EarlyStopping(
            monitor='val_accuracy',
            patience=10,
            restore_best_weights=True,
        )
    ]
)

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 4, 512)         │     2,623,488 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 256)            │       787,456 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 4)              │           260 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,453,380 (13.17 MB)

 Trainable params: 3,452,868 (13.17 MB)

 Non-trainable params: 512 (2.00 KB)

Epoch 1/50


73/73 ━━━━━━━━━━━━━━━━━━━━ 20s 133ms/step - accuracy: 0.2984 - f1_score: 0.2224 - loss: 1.5103 - weighted_accuracy: 0.2762 - val_accuracy: 0.2679 - val_f1_score: 0.1624 - val_loss: 1.3414 - val_weighted_accuracy: 0.2679 - learning_rate: 0.0010
Epoch 2/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 8s 103ms/step - accuracy: 0.2692 - f1_score: 0.2245 - loss: 1.6862 - weighted_accuracy: 0.3094 - val_accuracy: 0.2773 - val_f1_score: 0.1704 - val_loss: 1.3205 - val_weighted_accuracy: 0.2773 - learning_rate: 0.0010
Epoch 3/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 7s 99ms/step - accuracy: 0.2764 - f1_score: 0.2224 - loss: 1.3102 - weighted_accuracy: 0.3224 - val_accuracy: 0.1994 - val_f1_score: 0.1632 - val_loss: 1.3337 - val_weighted_accuracy: 0.1994 - learning_rate: 0.0010
Epoch 4/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 7s 89ms/step - accuracy: 0.2563 - f1_score: 0.2101 - loss: 1.3673 - weighted_accuracy: 0.2865 - val_accuracy: 0.2617 - val_f1_score: 0.1954 - val_loss: 1.4474 - val_weighted_accuracy: 0.2617 - learning_rate: 0.0

In [448]:
model_y1.evaluate(test_x, keras.utils.to_categorical(test_y1))

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.4145 - f1_score: 0.2297 - loss: 1.2748 - weighted_accuracy: 0.4145


[1.2365095615386963,
 0.42629483342170715,
 <tf.Tensor: shape=(4,), dtype=float32, numpy=array([0.19999997, 0.5773195 , 0.26229504, 0.        ], dtype=float32)>,
 0.42629483342170715,
 0.42629483342170715]

In [453]:
model_y1.save('/Users/khuzam/Desktop/fp/offence_model.keras')

In [452]:
ac= test_x[26].reshape((1, 4, 768))
model_y1.predict(ac)[0]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step


array([0.1746948 , 0.23655906, 0.26996866, 0.31877744], dtype=float32)

---

#### **Y2 classification**

In [156]:
keras.utils.clear_session()

model2= keras.models.Sequential([
    L.Input(input_shape),
    
    L.LSTM(units=512, dropout=0.4, recurrent_dropout=0.4, return_sequences=True),
    L.LSTM(units=256, dropout=0.4, recurrent_dropout=0.4, return_sequences=False),

    L.Dense(256, activation='sigmoid'),
    L.Dropout(0.2),
    L.Dense(units=8, activation='softmax')
])

model2.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 4, 512)         │     2,623,488 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 256)            │       787,456 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 8)              │         2,056 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,478,792 (13.27 MB)

 Trainable params: 3,478,792 (13.27 MB)

 Non-trainable params: 0 (0.00 B)

In [157]:
model2.compile(
    loss="categorical_crossentropy", 
    optimizer="adam",
    weighted_metrics=["accuracy"],
)

model2.fit(
    train_x,
    keras.utils.to_categorical(train_y2),
    epochs=50,
    batch_size=32,
    validation_data=[val_x, keras.utils.to_categorical(val_y2)],
    class_weight= class_weights2,
    callbacks= [
        keras.callbacks.ReduceLROnPlateau(
            patience=5,
            factor=0.1,
            monitor='val_accuracy',
            mode='min'
        ),
    ]
)

Epoch 1/50


73/73 ━━━━━━━━━━━━━━━━━━━━ 14s 110ms/step - accuracy: 0.1112 - loss: 2.2543 - val_accuracy: 0.0810 - val_loss: 2.0217 - learning_rate: 0.0010
Epoch 2/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 7s 93ms/step - accuracy: 0.1136 - loss: 2.1885 - val_accuracy: 0.2368 - val_loss: 1.9961 - learning_rate: 0.0010
Epoch 3/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 7s 95ms/step - accuracy: 0.1242 - loss: 2.1161 - val_accuracy: 0.0374 - val_loss: 2.1612 - learning_rate: 0.0010
Epoch 4/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 10s 96ms/step - accuracy: 0.1742 - loss: 2.0033 - val_accuracy: 0.1620 - val_loss: 2.0763 - learning_rate: 0.0010
Epoch 5/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 7s 94ms/step - accuracy: 0.1566 - loss: 2.0400 - val_accuracy: 0.0841 - val_loss: 2.0837 - learning_rate: 0.0010
Epoch 6/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 7s 95ms/step - accuracy: 0.1864 - loss: 2.0676 - val_accuracy: 0.2181 - val_loss: 1.9672 - learning_rate: 0.0010
Epoch 7/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 7s 99ms/step - accuracy: 0.2419 - loss: 1.9915 - val_accuracy: 0

In [159]:
model2.evaluate(test_x, keras.utils.to_categorical(test_y2))

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.1500 - loss: 2.0248


[2.049934148788452, 0.12350597977638245]

In [192]:
ac= test_x[0].reshape((1, 4, 768))
model2.predict(ac)[0]

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 663ms/step


array([6.1938804e-02, 2.4232332e-01, 4.7246044e-04, 9.5176227e-02,
       5.2352570e-04, 4.3035537e-02, 5.5631077e-01, 2.1936590e-04],
      dtype=float32)

In [266]:
keras.utils.clear_session()

inputs= L.Input((train_x.shape[1:]))

x = L.Conv1D(64, 1, activation='sigmoid')(inputs)
x = L.Conv1D(64, 1, activation='sigmoid')(x)
x = L.Conv1D(128, 1, activation='sigmoid')(x)
x = L.Conv1D(1024, 1, activation='sigmoid')(x)

x = L.GlobalMaxPooling1D()(x)

x = L.Dense(512, activation='sigmoid')(x)
x = L.Dense(256, activation='sigmoid')(x)
x = L.Dense(128, activation='relu')(x)
# x = L.Dense(64, activation='relu')(x)
outputs = L.Dense(8, activation='softmax')(x)

model= keras.models.Model(inputs=inputs, outputs=outputs)

model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 4, 768)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d (Conv1D)                 │ (None, 4, 64)          │        49,216 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 4, 64)          │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_2 (Conv1D)               │ (None, 4, 128)         │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_3 (Conv1D)               │ (None, 4, 1024)        │       132,096 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d            │ (None, 1024)           │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │       524,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 8)              │         1,032 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 883,848 (3.37 MB)

 Trainable params: 883,848 (3.37 MB)

 Non-trainable params: 0 (0.00 B)

In [267]:
model.compile(
    loss="categorical_crossentropy", 
    optimizer=keras.optimizers.Adam(learning_rate=0.0001, weight_decay=0.004),
    weighted_metrics=["accuracy"],
)

model.fit(
    train_x,
    keras.utils.to_categorical(train_y2),
    epochs=50,
    batch_size=32,
    validation_data=[val_x, keras.utils.to_categorical(val_y2)],
    class_weight= class_weights2,
    callbacks= [
        keras.callbacks.ReduceLROnPlateau(
            patience=5,
            factor=0.1,
            monitor='val_accuracy',
            mode='max'
        ),
        # keras.callbacks.EarlyStopping(
        #     monitor='val_loss',
        #     mode='max',
        #     patience=10,
        #     restore_best_weights=True,
        # )
    ]
)

Epoch 1/50


73/73 ━━━━━━━━━━━━━━━━━━━━ 5s 25ms/step - accuracy: 0.0962 - loss: 2.1375 - val_accuracy: 0.0467 - val_loss: 2.0728 - learning_rate: 1.0000e-04
Epoch 2/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.1018 - loss: 2.0802 - val_accuracy: 0.0436 - val_loss: 2.1409 - learning_rate: 1.0000e-04
Epoch 3/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.1129 - loss: 2.1410 - val_accuracy: 0.0374 - val_loss: 2.1322 - learning_rate: 1.0000e-04
Epoch 4/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.1380 - loss: 2.1862 - val_accuracy: 0.4112 - val_loss: 2.0333 - learning_rate: 1.0000e-04
Epoch 5/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 2s 29ms/step - accuracy: 0.1242 - loss: 2.0632 - val_accuracy: 0.0374 - val_loss: 2.0436 - learning_rate: 1.0000e-04
Epoch 6/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.1097 - loss: 2.2262 - val_accuracy: 0.1090 - val_loss: 1.9792 - learning_rate: 1.0000e-04
Epoch 7/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 2s 21ms/step - accuracy: 0.0936 - loss: 2.2

KeyboardInterrupt: 

---

### search

expirement with different hyperparameters

In [16]:
keras.utils.clear_session()

def build_model(scale=1, activation='relu', lr=0.001):
    m = keras.models.Sequential([
        L.Input(input_shape,),
        L.LSTM(units= 80 * scale, return_sequences=True),
        L.BatchNormalization(),
        L.LSTM(units=120 * scale),
        L.BatchNormalization(),
        L.Dropout(0.2),
        L.Dense(60 * scale, activation=activation),
        L.Dense(30 * scale, activation=activation),
        L.Dropout(0.2),
        L.Dense(units=4, activation='softmax')
    ])

    m.compile(
        loss="categorical_crossentropy", 
        metrics=[
            "accuracy",
            "f1_score",  
        ],
        optimizer=keras.optimizers.Adam(learning_rate=lr),  
    )

    return m

In [17]:
import keras_tuner

# keras.utils.clear_session()

def kt_build_model(hp: keras_tuner.HyperParameters):
    return build_model(
        scale=hp.Choice("scale", [1, 2, 3]),
        activation=hp.Choice("activation", ['relu', 'sigmoid']),
        lr=hp.Choice("lr", [0.00001, 0.0001, 0.001, 0.0002, 0.002, 0.005]),
    )

tuner = keras_tuner.Hyperband(
    kt_build_model,
    objective='accuracy',
    max_epochs=100,
)

tuner.search(
    train_x,
    keras.utils.to_categorical(train_y1),
    batch_size=16,
    validation_data=[val_x, keras.utils.to_categorical(val_y1)],
    class_weight= class_weights,
)

tuned_model = tuner.get_best_models()[0]
tuned_model.summary()

# tuner.results_summary()

Trial 36 Complete [00h 00m 19s]
accuracy: 0.2565761208534241

Best accuracy So Far: 0.43510133028030396
Total elapsed time: 00h 10m 04s


/Users/khuzam/Library/Python/3.11/lib/python/site-packages/keras/src/saving/saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 34 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 4, 80)          │       271,680 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 4, 80)          │           320 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 120)            │        96,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 120)            │           480 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 120)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 60)             │         7,260 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 30)             │         1,830 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 30)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 4)              │           124 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 378,174 (1.44 MB)

 Trainable params: 377,774 (1.44 MB)

 Non-trainable params: 400 (1.56 KB)

In [25]:
tuner.get_best_hyperparameters()

In [26]:
tuned_model.evaluate(test_x, keras.utils.to_categorical(test_y1))

8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.4524 - f1_score: 0.2199 - loss: 1.3405


[1.3373030424118042,
 0.4780876636505127,
 <tf.Tensor: shape=(4,), dtype=float32, numpy=array([0.        , 0.6645962 , 0.21428567, 0.09090906], dtype=float32)>]

In [27]:
ac= test_x[0].reshape((1, 4, 768))
tuned_model.predict(ac)[0]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 479ms/step


array([0.23138502, 0.28537822, 0.24720904, 0.23602773], dtype=float32)

---

## **Classification Using 1 models**

### Combine Y1 and Y2

#### Data

In [14]:
train_y= {
    'offence': keras.utils.to_categorical(train_y1),
    'foul_type': keras.utils.to_categorical(train_y2),
}

In [15]:
val_y= {
    'offence': keras.utils.to_categorical(val_y1),
    'foul_type': keras.utils.to_categorical(val_y2)
}

In [16]:
test_y= {
    'offence': keras.utils.to_categorical(test_y1),
    'foul_type': keras.utils.to_categorical(test_y2)
}

In [17]:
weights= {
    'offence': class_weights,
    'foul_type': class_weights2
}

---

### class weights

In [72]:
class_weights

{0: 1.9070723684210527,
 1: 0.4449347659247889,
 2: 0.8463503649635037,
 3: 21.47222222222222}

In [18]:
from functools import partial
import tensorflow as tf


def weighted_categorical_crossentropy(target, output, weights_table):
    weights_vect = weights_table.lookup(keras.ops.argmax(target, axis=-1))
    loss = keras.losses.categorical_crossentropy(target, output) * weights_vect # or (y_pred, y_true, sample_weight=weight_sample_class)
    # return keras.ops.mean(loss)
    return loss


# transform dictionary of weights into lookup table that can be used
def to_lookup_table(dictionary):
    return tf.lookup.StaticHashTable(
        tf.lookup.KeyValueTensorInitializer(
            list(dictionary.keys()), #HERE
            list(dictionary.values()), #HERE
            key_dtype=tf.int32,
            value_dtype=tf.float32,
        ),
        default_value=-1)

In [19]:
crossentropy_1 = partial(weighted_categorical_crossentropy, weights_table=to_lookup_table(class_weights))
crossentropy_2 = partial(weighted_categorical_crossentropy, weights_table=to_lookup_table(class_weights2))

### m1

In [209]:
keras.utils.clear_session()


inputs= L.Input(input_shape)
    
x = L.SimpleRNN(units=256, return_sequences=False)(inputs)
x= L.Dropout(0.2)(x)

x = L.Dense(units=128, activation='relu')(x)
x= L.BatchNormalization()(x)
x = L.Dense(units=32, activation='relu')(x)
x= L.BatchNormalization()(x)
x = L.Dense(units=16, activation='relu')(x)
x= L.BatchNormalization()(x)

offence= L.Dense(units=4, activation='softmax', name='offence')(x)
foul_type= L.Dense(units=8, activation='softmax', name='foul_type')(x)

model_y= keras.models.Model(inputs=inputs, outputs={
    'offence': offence,
    'foul_type': foul_type,
})

In [210]:
# keras.utils.clear_session()


# inputs= L.Input(input_shape,)
    
# x= L.LSTM(units= 80, return_sequences=True)(inputs)
# x= L.BatchNormalization()(x)
# x= L.LSTM(units=120)(x)
# x= L.BatchNormalization()(x)
# x= L.Dropout(0.2)(x)
# x= L.Dense(60, activation='relu')(x)
# x= L.Dense(30, activation='relu')(x)
# x= L.Dropout(0.2)(x)
# offence= L.Dense(units=4, activation='softmax', name='offence')(x)
# foul_type= L.Dense(units=8, activation='softmax', name='foul_type')(x)

# model_y= keras.models.Model(inputs=inputs, outputs={
#     'offence': offence,
#     'foul_type': foul_type,
# })

model_y.summary()

model_y.compile(
    optimizer="adam", 
    loss={
        'offence':  crossentropy_1,
        'foul_type': crossentropy_2,
    }, 
    # metrics={
    #     'offence': ["accuracy", "f1_score"],
    #     'foul_type': ["accuracy", "f1_score"],
    # },
    weighted_metrics={
        'offence': ["accuracy"],
        'foul_type': ["accuracy"],
    },
)

model_y.fit(
    train_x,
    y,
    epochs=50,
    batch_size=16,
    validation_data=[val_x, val],
    callbacks= [
        keras.callbacks.ReduceLROnPlateau(
            patience=5,
            factor=0.1,
            monitor='val_loss',
            mode='min'
        ),
        # keras.callbacks.EarlyStopping(
        #     monitor='val_offence_accuracy',
        #     mode='max',
        #     patience=10,
        #     restore_best_weights=True,
        # )
    ]
)

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 4, 768)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ simple_rnn          │ (None, 256)       │    262,400 │ input_layer[0][0] │
│ (SimpleRNN)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 256)       │          0 │ simple_rnn[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 128)       │     32,896 │ dropout[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 128)       │        512 │ dense[0][0]       │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 32)        │      4,128 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 32)        │        128 │ dense_1[0][0]     │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 16)        │        528 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 16)        │         64 │ dense_2[0][0]     │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ foul_type (Dense)   │ (None, 8)         │        136 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ offence (Dense)     │ (None, 4)         │         68 │ batch_normalizat… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 300,860 (1.15 MB)

 Trainable params: 300,508 (1.15 MB)

 Non-trainable params: 352 (1.38 KB)

Epoch 1/50


145/145 ━━━━━━━━━━━━━━━━━━━━ 9s 13ms/step - foul_type_accuracy: 0.1413 - loss: 4.3533 - offence_accuracy: 0.2471 - val_foul_type_accuracy: 0.2555 - val_loss: 3.8310 - val_offence_accuracy: 0.2773 - learning_rate: 0.0010
Epoch 2/50
145/145 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - foul_type_accuracy: 0.1449 - loss: 3.7465 - offence_accuracy: 0.2330 - val_foul_type_accuracy: 0.2305 - val_loss: 3.7757 - val_offence_accuracy: 0.1682 - learning_rate: 0.0010
Epoch 3/50
145/145 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - foul_type_accuracy: 0.1571 - loss: 3.3999 - offence_accuracy: 0.2326 - val_foul_type_accuracy: 0.1028 - val_loss: 3.6084 - val_offence_accuracy: 0.0966 - learning_rate: 0.0010
Epoch 4/50
145/145 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - foul_type_accuracy: 0.1441 - loss: 3.5997 - offence_accuracy: 0.2726 - val_foul_type_accuracy: 0.0685 - val_loss: 3.4227 - val_offence_accuracy: 0.3240 - learning_rate: 0.0010
Epoch 5/50
145/145 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - foul_type_accuracy: 0.1242 - loss: 

In [211]:
model_y.evaluate(test_x, test_y)

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - foul_type_accuracy: 0.1243 - loss: 4.1010 - offence_accuracy: 0.3813 


[3.774942636489868, 0.131474107503891, 0.394422322511673]

In [212]:
ac= test_x[0].reshape((1, 4, 768))
print(model_y.predict(ac)['offence'][0].round(4))
print(model_y.predict(ac)['foul_type'][0].round(3))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 426ms/step
[0.1687 0.4388 0.3645 0.028 ]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
[0.085 0.095 0.071 0.109 0.165 0.232 0.178 0.066]


In [174]:
ac= test_x[22].reshape((1, 4, 768))

In [175]:
print(model_y.predict(ac)['offence'][0].round(3))
print(model_y.predict(ac)['foul_type'][0].round(3))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
[0.142 0.148 0.181 0.528]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
[0.211 0.055 0.092 0.225 0.17  0.089 0.102 0.055]


---

### m4

In [239]:
keras.utils.clear_session()

inputs= L.Input(input_shape)

x = L.GRU(units=265, return_sequences=True)(inputs)
x= L.Dropout(0.2)(x)
x = L.GRU(units=512, return_sequences=True)(x)
x= L.Dropout(0.2)(x)
x = L.GRU(units=1024, return_sequences=False)(x)
x= L.Dropout(0.2)(x)

x = L.Dense(units=512, activation='relu')(x)
x = L.Dense(units=128, activation='relu')(x)
x = L.Dense(units=64, activation='relu')(x)
x = L.Dense(units=32, activation='relu')(x)
x = L.Dense(units=16, activation='relu')(x)

offence= L.Dense(units=4, activation='softmax', name='offence')(x)
foul_type= L.Dense(units=8, activation='softmax', name='foul_type')(x)

m= keras.models.Model(inputs=inputs, outputs={
    'offence': offence,
    'foul_type': foul_type,
})

m.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 4, 768)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ gru (GRU)           │ (None, 4, 265)    │    822,825 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 4, 265)    │          0 │ gru[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ gru_1 (GRU)         │ (None, 4, 512)    │  1,196,544 │ dropout[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 4, 512)    │          0 │ gru_1[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ gru_2 (GRU)         │ (None, 1024)      │  4,724,736 │ dropout_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 1024)      │          0 │ gru_2[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 512)       │    524,800 │ dropout_2[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 512)       │      2,048 │ dense[0][0]       │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 128)       │     65,664 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128)       │        512 │ dense_1[0][0]     │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 64)        │      8,256 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 64)        │        256 │ dense_2[0][0]     │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 32)        │      2,080 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 32)        │        128 │ dense_3[0][0]     │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 16)        │        528 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 16)        │         64 │ dense_4[0][0]     │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ foul_type (Dense)   │ (None, 8)         │        136 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ offence (Dense)     │ (None, 4)         │         68 │ batch_normalizat… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 7,348,645 (28.03 MB)

 Trainable params: 7,347,141 (28.03 MB)

 Non-trainable params: 1,504 (5.88 KB)

In [ ]:
m.compile(
    optimizer="adam", 
    loss={
        'offence':  crossentropy_1,
        'foul_type': crossentropy_2,
    }, 
    weighted_metrics={
        'offence': ["accuracy"],
        'foul_type': ["accuracy"],
    },
)

m.fit(
    train_x,
    y,
    epochs=50,
    batch_size=32,
    validation_data=[val_x, val],
    callbacks= [
        keras.callbacks.ReduceLROnPlateau(
            patience=5,
            factor=0.1,
            monitor='val_loss',
            mode='min'
        ),
    ]
)

---

### keep


In [17]:
input_shape= train_x.shape[1:]

In [37]:
keras.utils.clear_session()


inputs= L.Input(input_shape,)
    
x= L.LSTM(units=256, dropout=0.4, recurrent_dropout=0.4, return_sequences=True)(inputs)
x= L.LSTM(units=128, dropout=0.4, recurrent_dropout=0.4, return_sequences=False)(x)
# x= L.Dropout(0.2)(x)

x= L.Dense(128, activation='relu')(x)
x= L.Dense(56, activation='relu')(x)
x= L.Dense(32, activation='relu')(x)
x= L.Dropout(0.2)(x)
offence= L.Dense(units=, activation='relu')(x)
offence= L.Dense(units=4, activation='softmax', name='offence')(x)
foul_type= L.Dense(units=8, activation='softmax', name='foul_type')(x)

model_y= keras.models.Model(inputs=inputs, outputs={
    'offence': offence,
    'foul_type': foul_type,
})

In [38]:
model_y.summary()

model_y.compile(
    optimizer="adam", 
    loss={
        'offence':  crossentropy_1,
        'foul_type': crossentropy_2,
    }, 
    weighted_metrics={
        'offence': ["accuracy"],
        'foul_type': ["accuracy"],
    },
)

model_y.fit(
    train_x,
    train_y,
    epochs=50,
    batch_size=16,
    validation_data=[val_x, val_y],
    callbacks= [
        keras.callbacks.ReduceLROnPlateau(
            patience=5,
            factor=0.1,
            monitor='val_loss',
            mode='min'
        ),
        keras.callbacks.EarlyStopping(
            monitor='val_foul_type_accuracy',
            mode='max',
            patience=15,
            restore_best_weights=True,
        )
    ]
)

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 4, 768)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ (None, 4, 256)    │  1,049,600 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_1 (LSTM)       │ (None, 128)       │    197,120 │ lstm[0][0]        │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 128)       │     16,512 │ lstm_1[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 128)       │        512 │ dense[0][0]       │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 56)        │      7,224 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 56)        │        224 │ dense_1[0][0]     │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 32)        │      1,824 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 32)        │        128 │ dense_2[0][0]     │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 32)        │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ foul_type (Dense)   │ (None, 8)         │        264 │ dropout[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ offence (Dense)     │ (None, 4)         │        132 │ dropout[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 1,273,540 (4.86 MB)

 Trainable params: 1,273,108 (4.86 MB)

 Non-trainable params: 432 (1.69 KB)

Epoch 1/50


145/145 ━━━━━━━━━━━━━━━━━━━━ 11s 34ms/step - foul_type_accuracy: 0.1121 - loss: 4.4584 - offence_accuracy: 0.2092 - val_foul_type_accuracy: 0.1246 - val_loss: 3.3507 - val_offence_accuracy: 0.3115 - learning_rate: 0.0010
Epoch 2/50
145/145 ━━━━━━━━━━━━━━━━━━━━ 5s 34ms/step - foul_type_accuracy: 0.1126 - loss: 4.1913 - offence_accuracy: 0.2023 - val_foul_type_accuracy: 0.3240 - val_loss: 3.6797 - val_offence_accuracy: 0.4548 - learning_rate: 0.0010
Epoch 3/50
145/145 ━━━━━━━━━━━━━━━━━━━━ 5s 31ms/step - foul_type_accuracy: 0.1135 - loss: 3.7138 - offence_accuracy: 0.2248 - val_foul_type_accuracy: 0.1028 - val_loss: 3.5314 - val_offence_accuracy: 0.1184 - learning_rate: 0.0010
Epoch 4/50
145/145 ━━━━━━━━━━━━━━━━━━━━ 4s 31ms/step - foul_type_accuracy: 0.1093 - loss: 3.7568 - offence_accuracy: 0.2284 - val_foul_type_accuracy: 0.0841 - val_loss: 3.3648 - val_offence_accuracy: 0.2617 - learning_rate: 0.0010
Epoch 5/50
145/145 ━━━━━━━━━━━━━━━━━━━━ 5s 36ms/step - foul_type_accuracy: 0.0941 - lo

In [39]:
model_y.evaluate(test_x, test_y)

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - foul_type_accuracy: 0.2737 - loss: 4.0740 - offence_accuracy: 0.4753


[3.785036087036133, 0.3187251091003418, 0.5059760808944702]

---

### Best found through search search

In [225]:
train_x_2= train_x[:,1:,:]
val_x_2= val_x[:,1:,:]
test_x_2= test_x[:,1:,:]

In [439]:
keras.utils.clear_session()

def build_model(scale=1, activation='relu', lr=0.001):
    inputs= L.Input(input_shape,)
    
    x= L.LSTM(units=512*scale, dropout=0.4, recurrent_dropout=0.4, return_sequences=True)(inputs)
    x= L.BatchNormalization()(x)
    x= L.LSTM(units=256*scale, dropout=0.4, recurrent_dropout=0.4, return_sequences=False)(x)
    # x= L.Dropout(0.2)(x)

    x= L.Dense(128*scale, activation='relu')(x)
    # x= L.Dropout(0.2)(x)

    h1 = L.Dense(units=256, activation=activation)(x)
    offence= L.Dense(units=4, activation='softmax', name='offence')(h1)

    h2= L.Dense(units=256*scale, activation=activation)(x)
    h2= L.Dense(units=128*scale, activation=activation)(h2)
    h2= L.Dense(units=56*scale, activation=activation)(h2)
    foul_type= L.Dense(units=8, activation='softmax', name='foul_type')(h2)

    m= keras.models.Model(inputs=inputs, outputs={
        'offence': offence,
        'foul_type': foul_type,
    })

    m.compile(
        optimizer=keras.optimizers.Adam(learning_rate=lr), 
        loss={
            'offence':  crossentropy_1,
            'foul_type': crossentropy_2,
        }, 
        weighted_metrics={
            'offence': ["accuracy"],
            'foul_type': ["accuracy"],
        },  
    )

    return m

In [250]:
import keras_tuner

# keras.utils.clear_session()

def kt_build_model(hp: keras_tuner.HyperParameters):
    return build_model(
        scale=hp.Choice("scale", [1, 2]),
        activation=hp.Choice("activation", ['relu', 'sigmoid']),
        lr=hp.Choice("lr", [0.00001, 0.0001, 0.001, 0.0002, 0.002]),
    )

tuner = keras_tuner.Hyperband(
    kt_build_model,
    objective=keras_tuner.Objective("val_offence_accuracy", direction="max"),
    max_epochs=100,
)

tuner.search(
    train_x,
    train_y,
    batch_size=16,
    validation_data=[val_x, val_y],
)

tuned_model = tuner.get_best_models()[0]
tuned_model.summary()

# tuner.results_summary()

Trial 19 Complete [00h 00m 37s]
val_offence_accuracy: 0.09968847036361694

Best val_offence_accuracy So Far: 0.44859811663627625
Total elapsed time: 00h 18m 48s


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 4, 768)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ (None, 4, 512)    │  2,623,488 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 4, 512)    │      2,048 │ lstm[0][0]        │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_1 (LSTM)       │ (None, 256)       │    787,456 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 128)       │     32,896 │ lstm_1[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 256)       │     33,024 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 128)       │     32,896 │ dense_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 56)        │      7,224 │ dense_3[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 256)       │     33,024 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ foul_type (Dense)   │ (None, 8)         │        456 │ dense_4[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ offence (Dense)     │ (None, 4)         │      1,028 │ dense_1[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 3,553,540 (13.56 MB)

 Trainable params: 3,552,516 (13.55 MB)

 Non-trainable params: 1,024 (4.00 KB)

In [251]:
tuned_model.evaluate(test_x, test_y)

8/8 ━━━━━━━━━━━━━━━━━━━━ 3s 29ms/step - foul_type_accuracy: 0.0243 - loss: 3.8107 - offence_accuracy: 0.4890


[3.6010377407073975, 0.019920319318771362, 0.48605579137802124]

In [252]:
best_hp = tuner.get_best_hyperparameters()[0].values
best_hp

{'scale': 1,
 'activation': 'sigmoid',
 'lr': 0.001,
 'tuner/epochs': 2,
 'tuner/initial_epoch': 0,
 'tuner/bracket': 4,
 'tuner/round': 0}

In [255]:
ac= test_x[0].reshape((1, 4, 768))
print(tuned_model.predict(ac)['offence'][0].round(4))
print(tuned_model.predict(ac)['foul_type'][0].round(3))

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 941ms/step
[0.1996 0.3005 0.2425 0.2573]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
[0.11  0.111 0.128 0.128 0.101 0.124 0.139 0.159]


---

### Reload best model 

**note**: this was the model used for test_predictions.json

In [440]:
input_shape= train_x.shape[1:]

In [441]:
keras.utils.clear_session()
offence_model = build_model(scale=1, activation='sigmoid')
offence_model.load_weights('/Users/khuzam/Desktop/fp/weights.weights.h5')

In [442]:
offence_model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.0001, weight_decay=0.004), 
    loss={
        'offence':  crossentropy_1,
        'foul_type': crossentropy_2,
    }, 
    weighted_metrics={
        'offence': ["accuracy"],
        'foul_type': ["accuracy"],
    },
)

In [445]:
offence_model.evaluate(test_x, test_y)

ValueError: Creating variables on a non-first call to a function decorated with tf.function.

---

### get predicitions on test data

In [117]:
df= pd.read_csv('/Users/khuzam/Desktop/fp/SoccerNet/test_labels/test.csv')

In [147]:
print(best_model.predict(ac)['offence'].round(4))
print(best_model.predict(ac)['foul_type'][0].round(3))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
[[0.2476 0.2731 0.2285 0.2509]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
[0.149 0.154 0.124 0.136 0.088 0.149 0.104 0.096]


In [121]:
print(np.argmax(best_model.predict(ac)['offence'][0].round(4)))
print(np.argmax(best_model.predict(ac)['foul_type'][0].round(3)))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1


In [256]:
actions_ints= np.load('/Users/khuzam/Desktop/fp/SoccerNet/test_labels/number_of_actions.npy')

In [257]:
#for every action in action_ints
def get_predictions(model, x):
    #predictions dictionary
    test_predictions= {
        "Actions": {
            
        }
    }

    for i, action in enumerate(actions_ints):

        #predict
        ac= x[i].reshape((1, 4, 768))
        output = model.predict(ac)
        offence_pred= np.argmax(output['offence'][0].round(4))
        type_pred= np.argmax(output['foul_type'][0].round(3))
        
        action_number= str(action)
        offence=''
        severity=''

        #divide offence/severity
        if offence_pred==0:
            offence= 'No offence'
        elif offence_pred==1:
            offence= 'Offence'
            severity= '1.0'
        elif offence_pred==2:
            offence= 'Offence'
            severity= '3.0'
        elif offence_pred==3:
            offence= 'Offence'
            severity= '5.0'

        #insert into dictionary
        test_predictions['Actions'][action_number]= {
            'Action class': action_class[type_pred],
            'Offence': offence,
            'Severity': severity
        }

    #save json file
    with open("/Users/khuzam/Desktop/fp/SoccerNet/test_predictions3.json", "w") as outfile: 
        json.dump(test_predictions, outfile)


In [258]:
get_predictions(tuned_model, test_x)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━

---

### predict using seperate models

In [167]:
model_offence= model_y1
model_foul_type= keras.models.load_model('/Users/khuzam/Desktop/fp/tunedmodel.keras')

/Users/khuzam/Library/Python/3.11/lib/python/site-packages/keras/src/saving/saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'adam', because it has 34 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [177]:
model_offence.evaluate(test_x, keras.utils.to_categorical(test_y1))

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.4226 - f1_score: 0.2618 - loss: 1.2051 - weighted_accuracy: 0.4226


[1.186476230621338,
 0.38645419478416443,
 <tf.Tensor: shape=(4,), dtype=float32, numpy=array([0.20895518, 0.48669195, 0.31515142, 0.        ], dtype=float32)>,
 0.38645419478416443,
 0.38645419478416443]

In [179]:
model_foul_type.evaluate(test_x, keras.utils.to_categorical(test_y2))

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.3838 - loss: 1.6735


[1.6309515237808228, 0.42629483342170715]

In [175]:
ac= test_x[4].reshape((1, 4, 768))
model_foul_type.predict(ac)[0]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step


array([0.1843036 , 0.42308518, 0.03865341, 0.13094133, 0.03425165,
       0.05003945, 0.12312968, 0.01559575], dtype=float32)

In [172]:
action_class

{0: 'Tackling',
 1: 'Standing tackling',
 2: 'High leg',
 3: 'Holding',
 4: 'Pushing',
 5: 'Elbowing',
 6: 'Challenge',
 7: 'Dive',
 8: 'Dont know'}

In [170]:
#predictions dictionary
test_predictions= {
    "Actions": {
        
    }
}

for i, action in enumerate(actions_ints):

    #predict
    ac= test_x[i].reshape((1, 4, 768))
    offence_pred= np.argmax(model_offence.predict(ac)[0].round(4))
    type_pred= np.argmax(model_foul_type.predict(ac)[0].round(3))
    
    action_number= str(action)
    offence=''
    severity=''

    #divide offence/severity
    if offence_pred==0:
        offence= 'No offence'
    elif offence_pred==1:
        offence= 'Offence'
        severity= '1.0'
    elif offence_pred==2:
        offence= 'Offence'
        severity= '3.0'
    elif offence_pred==3:
        offence= 'Offence'
        severity= '5.0'

    #insert into dictionary
    test_predictions['Actions'][action_number]= {
        'Action class': action_class[type_pred],
        'Offence': offence,
        'Severity': severity
    }

#save json file
with open("/Users/khuzam/Desktop/fp/SoccerNet/test_predictions2.json", "w") as outfile: 
    json.dump(test_predictions, outfile)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 210ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━

---
## **Classification Using 2 Seperate MultiView models**

**Final Models**

In [64]:
multi_train_x= train_x.reshape(-1, train_x.shape[-1])
multi_train_y1= train_y1.repeat(train_x.shape[1])
multi_train_y2= train_y2.repeat(train_x.shape[1])
multi_val_x= val_x.reshape(-1, val_x.shape[-1])
multi_val_y1= val_y1.repeat(val_x.shape[1])
multi_val_y2= val_y2.repeat(val_x.shape[1])
multi_test_x= test_x.reshape(-1, test_x.shape[-1])
multi_test_y1= test_y1.repeat(test_x.shape[1])
multi_test_y2= test_y2.repeat(test_x.shape[1])

In [15]:
#Calculate class weights
multi_class_weights1 = compute_class_weight(class_weight="balanced", classes=np.unique(multi_train_y1), y=multi_train_y1)
multi_class_weights1 = dict(zip(range(len(multi_class_weights1)), multi_class_weights1.tolist()))

In [16]:
#Calculate class weights
multi_class_weights2 = compute_class_weight(class_weight="balanced", classes=np.unique(multi_train_y2), y=multi_train_y2)
multi_class_weights2 = dict(zip(range(len(multi_class_weights2)), multi_class_weights2.tolist()))

### multi view Y2

**FINAL MODEL**

In [23]:
keras.utils.clear_session()

input_shape= multi_test_x.shape[1:]

inputs= L.Input(input_shape)

x= L.Dense(256, activation='relu')(inputs)
x= L.Dense(128, activation='relu')(x)
x= L.Dense(64, activation='relu')(x)
x= L.Dense(32, activation='relu')(x)
outputs= L.Dense(8, activation='softmax')(x)

model= keras.models.Model(inputs=inputs, outputs=outputs)

model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 768)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       196,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 8)              │           264 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 240,360 (938.91 KB)

 Trainable params: 240,360 (938.91 KB)

 Non-trainable params: 0 (0.00 B)

In [24]:
model.compile(
    loss="categorical_crossentropy", 
    optimizer=keras.optimizers.Adam(learning_rate=0.0001, weight_decay=0.004),
    metrics=['accuracy'],
    weighted_metrics=["accuracy"],
)

model.fit(
    multi_train_x,
    keras.utils.to_categorical(multi_train_y2),
    epochs=100,
    batch_size=32,
    validation_data=[multi_val_x, keras.utils.to_categorical(multi_val_y2)],
    class_weight= multi_class_weights2,
    callbacks= [
        keras.callbacks.ReduceLROnPlateau(
            patience=5,
            factor=0.1,
            monitor='val_loss',
            mode='min'
        ),
        keras.callbacks.EarlyStopping(
            monitor='val_accuracy',
            mode='max',
            patience=10,
            restore_best_weights=True,
        )
    ]
)

Epoch 1/100
290/290 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.1188 - loss: 2.1511 - weighted_accuracy: 0.1444 - val_accuracy: 0.1386 - val_loss: 2.0577 - val_weighted_accuracy: 0.1386 - learning_rate: 1.0000e-04
Epoch 2/100
290/290 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.1532 - loss: 2.0383 - weighted_accuracy: 0.2061 - val_accuracy: 0.1207 - val_loss: 2.0842 - val_weighted_accuracy: 0.1207 - learning_rate: 1.0000e-04
Epoch 3/100
290/290 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.1459 - loss: 1.9675 - weighted_accuracy: 0.2559 - val_accuracy: 0.1581 - val_loss: 2.0357 - val_weighted_accuracy: 0.1581 - learning_rate: 1.0000e-04
Epoch 4/100
290/290 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.1839 - loss: 1.9012 - weighted_accuracy: 0.2916 - val_accuracy: 0.1207 - val_loss: 2.0616 - val_weighted_accuracy: 0.1207 - learning_rate: 1.0000e-04
Epoch 5/100
290/290 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.2124 - loss: 1.8198 - weighted_accuracy: 0.3306 - val_accuracy: 0

In [26]:
model.evaluate(multi_test_x, keras.utils.to_categorical(multi_test_y2))

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1793 - loss: 1.9792 - weighted_accuracy: 0.1793


[1.97796630859375,
 0.18625497817993164,
 0.18625497817993164,
 0.18625497817993164]

In [28]:
r= model.predict(test_x[0])
r

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 146ms/step


array([[0.11381573, 0.12139218, 0.15151156, 0.11765604, 0.15514587,
        0.10438866, 0.12215182, 0.11393809],
       [0.11917346, 0.10505199, 0.11024835, 0.1401129 , 0.15353177,
        0.12130456, 0.12781791, 0.12275903],
       [0.11917346, 0.10505199, 0.11024835, 0.1401129 , 0.15353177,
        0.12130456, 0.12781791, 0.12275903],
       [0.11917346, 0.10505199, 0.11024835, 0.1401129 , 0.15353177,
        0.12130456, 0.12781791, 0.12275903]], dtype=float32)

In [29]:
model.save('./multi_foul_model.keras')

### multi view Y1

In [15]:
keras.utils.clear_session()

input_shape= multi_test_x.shape[1:]

inputs= L.Input(input_shape)

x= L.Dense(256, activation='sigmoid')(inputs)
x= L.Dense(128, activation='sigmoid')(x)
x= L.Dense(64, activation='sigmoid')(x)
outputs= L.Dense(4, activation='softmax')(x)

offence_model= keras.models.Model(inputs=inputs, outputs=outputs)

offence_model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 768)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       196,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 4)              │           260 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 238,276 (930.77 KB)

 Trainable params: 238,276 (930.77 KB)

 Non-trainable params: 0 (0.00 B)

In [16]:
offence_model.compile(
    loss="categorical_crossentropy", 
    optimizer=keras.optimizers.Adam(learning_rate=0.0001, weight_decay=0.004),
    metrics=['accuracy'],
    weighted_metrics=["accuracy"],
)

offence_model.fit(
    multi_train_x,
    keras.utils.to_categorical(multi_train_y1),
    epochs=100,
    batch_size=32,
    validation_data=[multi_val_x, keras.utils.to_categorical(multi_val_y1)],
    class_weight= multi_class_weights1,
    callbacks= [
        keras.callbacks.ReduceLROnPlateau(
            patience=5,
            factor=0.1,
            monitor='val_loss',
            mode='min'
        ),
        keras.callbacks.EarlyStopping(
            monitor='val_accuracy',
            mode='max',
            patience=10,
            restore_best_weights=True,
        )
    ]
)

Epoch 1/100
290/290 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.3167 - loss: 1.4311 - weighted_accuracy: 0.2577 - val_accuracy: 0.1955 - val_loss: 1.3898 - val_weighted_accuracy: 0.1955 - learning_rate: 1.0000e-04
Epoch 2/100
290/290 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.1443 - loss: 1.4248 - weighted_accuracy: 0.2508 - val_accuracy: 0.4377 - val_loss: 1.3518 - val_weighted_accuracy: 0.4377 - learning_rate: 1.0000e-04
Epoch 3/100
290/290 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.3556 - loss: 1.3778 - weighted_accuracy: 0.2649 - val_accuracy: 0.2368 - val_loss: 1.3965 - val_weighted_accuracy: 0.2368 - learning_rate: 1.0000e-04
Epoch 4/100
290/290 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.2622 - loss: 1.4003 - weighted_accuracy: 0.3037 - val_accuracy: 0.2321 - val_loss: 1.3397 - val_weighted_accuracy: 0.2321 - learning_rate: 1.0000e-04
Epoch 5/100
290/290 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.3125 - loss: 1.4100 - weighted_accuracy: 0.2921 - val_accuracy: 0

In [17]:
offence_model.evaluate(multi_test_x, keras.utils.to_categorical(multi_test_y1))

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4602 - loss: 1.3464 - weighted_accuracy: 0.4602


[1.3429158926010132,
 0.4711155295372009,
 0.4711155295372009,
 0.4711155295372009]

In [40]:
r= offence_model.predict(test_x[0])
r

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step


array([[0.21983105, 0.25222132, 0.26474062, 0.26320705],
       [0.22829409, 0.27656996, 0.21986678, 0.27526918],
       [0.22829409, 0.27656996, 0.21986678, 0.27526918],
       [0.22829409, 0.27656996, 0.21986678, 0.27526918]], dtype=float32)

In [18]:
offence_model.save('./multi_offence_model.keras')

---

### prediction using multiview

In [19]:
actions_ints= np.load('./labels/test_labels/number_of_actions.npy')

In [20]:
offence_model= keras.models.load_model('./multi_offence_model.keras')
foul_model= keras.models.load_model('./multi_foul_model.keras')

In [21]:
offence_model.predict(test_x[0])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step


array([[0.2309632 , 0.25006244, 0.25714484, 0.26182956],
       [0.22715802, 0.28345492, 0.2272469 , 0.26214018],
       [0.22715802, 0.28345492, 0.2272469 , 0.26214018],
       [0.22715802, 0.28345492, 0.2272469 , 0.26214018]], dtype=float32)

In [22]:
#for every 4 clips
#model.predict
#get argmax of every clip
#get max of all clips
#prediction= max

#predictions dictionary
test_predictions= {
    "Actions": {
        
    }
}

for i, action in enumerate(actions_ints):

    #predict
    ac= test_x[i].reshape((1, 4, 768))
    offence_pred= offence_model.predict(test_x[i])
    type_pred= foul_model.predict(test_x[i])

    y1= []
    for i, el in enumerate(offence_pred):
        y1.append(np.argmax(offence_pred[i]))

    y2= []
    for i, el in enumerate(type_pred):
        y2.append(np.argmax(type_pred[i]))

    majority_y1= np.asarray(y1).max()
    majority_y2= np.asarray(y2).max()
    
    action_number= str(action)
    offence=''
    severity=''

    #divide offence/severity
    if majority_y1==0:
        offence= 'No offence'
    elif majority_y1==1:
        offence= 'Offence'
        severity= '1.0'
    elif majority_y1==2:
        offence= 'Offence'
        severity= '3.0'
    elif majority_y1==3:
        offence= 'Offence'
        severity= '5.0'

    #insert into dictionary
    test_predictions['Actions'][action_number]= {
        'Action class': action_class[majority_y2],
        'Offence': offence,
        'Severity': severity
    }

#save json file
with open("/Users/khuzam/Desktop/fp/SoccerNet/multi_test_predictions.json", "w") as outfile: 
    json.dump(test_predictions, outfile)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━

---

In [28]:
a= np.zeros((2, 100))
b= np.zeros((2, 100))
x= np.concatenate((a, b), axis=0)

In [59]:
x= np.asarray(np.concatenate((multi_train_x, multi_val_x), axis=0))
y1= np.asarray(np.concatenate((multi_train_y1, multi_val_y1), axis=0))
y2= np.asarray(np.concatenate((multi_train_y2, multi_val_y2), axis=0))

y2

In [73]:
keras.utils.clear_session()

input_shape= multi_test_x.shape[1:]

inputs= L.Input(input_shape)

x= L.Dense(256, activation='relu')(inputs)
x= L.Dense(128, activation='relu')(x)
x= L.Dense(64, activation='relu')(x)
x= L.Dense(32, activation='relu')(x)
outputs= L.Dense(8, activation='softmax')(x)

model= keras.models.Model(inputs=inputs, outputs=outputs)

model.compile(
    loss="categorical_crossentropy", 
    optimizer=keras.optimizers.Adam(learning_rate=0.0001, weight_decay=0.004),
    metrics=['accuracy'],
    weighted_metrics=["accuracy"],
)

In [74]:
model.fit(
    multi_train_x,
    keras.utils.to_categorical(multi_train_y2),
    epochs=100,
    batch_size=32,
    # validation_data=[multi_val_x, keras.utils.to_categorical(multi_val_y2)],
    validation_split=0.2,
    class_weight= multi_class_weights2,
    callbacks= [
        keras.callbacks.ReduceLROnPlateau(
            patience=5,
            factor=0.1,
            monitor='val_loss',
            mode='min'
        ),
        keras.callbacks.EarlyStopping(
            monitor='val_accuracy',
            mode='max',
            patience=10,
            restore_best_weights=True,
        )
    ]
)
model.fit(
    multi_train_x,
    keras.utils.to_categorical(multi_train_y2),
    epochs=100,
    batch_size=32,
    # validation_data=[multi_val_x, keras.utils.to_categorical(multi_val_y2)],
    validation_split=0.2,
    class_weight= multi_class_weights2,
    callbacks= [
        keras.callbacks.ReduceLROnPlateau(
            patience=5,
            factor=0.1,
            monitor='val_loss',
            mode='min'
        ),
        keras.callbacks.EarlyStopping(
            monitor='val_accuracy',
            mode='max',
            patience=10,
            restore_best_weights=True,
        )
    ]
)
model.fit(
    multi_test_x,
    keras.utils.to_categorical(multi_test_y2),
    epochs=100,
    batch_size=32,
    # validation_data=[multi_val_x, keras.utils.to_categorical(multi_val_y2)],
    validation_split=0.2,
    class_weight= multi_class_weights2,
    callbacks= [
        keras.callbacks.ReduceLROnPlateau(
            patience=5,
            factor=0.1,
            monitor='val_loss',
            mode='min'
        ),
        keras.callbacks.EarlyStopping(
            monitor='val_accuracy',
            mode='max',
            patience=10,
            restore_best_weights=True,
        )
    ]
)

Epoch 1/100


232/232 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.1300 - loss: 2.0621 - weighted_accuracy: 0.1407 - val_accuracy: 0.1056 - val_loss: 2.0530 - val_weighted_accuracy: 0.1056 - learning_rate: 1.0000e-04
Epoch 2/100
232/232 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.1411 - loss: 2.0610 - weighted_accuracy: 0.2439 - val_accuracy: 0.1067 - val_loss: 2.0403 - val_weighted_accuracy: 0.1067 - learning_rate: 1.0000e-04
Epoch 3/100
232/232 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.1654 - loss: 1.9604 - weighted_accuracy: 0.2940 - val_accuracy: 0.1401 - val_loss: 2.0420 - val_weighted_accuracy: 0.1401 - learning_rate: 1.0000e-04
Epoch 4/100
232/232 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.2185 - loss: 1.9019 - weighted_accuracy: 0.3121 - val_accuracy: 0.1045 - val_loss: 2.0495 - val_weighted_accuracy: 0.1045 - learning_rate: 1.0000e-04
Epoch 5/100
232/232 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.2371 - loss: 1.7952 - weighted_accuracy: 0.3716 - val_accuracy: 0.1654 - val_

In [76]:
model.evaluate(multi_test_x, keras.utils.to_categorical(multi_test_y2))

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1599 - loss: 1.9990 - weighted_accuracy: 0.1599


[1.9917044639587402,
 0.18924303352832794,
 0.18924303352832794,
 0.18924303352832794]

In [77]:
keras.utils.clear_session()

input_shape= multi_test_x.shape[1:]

inputs= L.Input(input_shape)

x= L.Dense(256, activation='sigmoid')(inputs)
x= L.Dense(128, activation='sigmoid')(x)
x= L.Dense(64, activation='sigmoid')(x)
outputs= L.Dense(4, activation='softmax')(x)

offence_model= keras.models.Model(inputs=inputs, outputs=outputs)


In [78]:
offence_model.compile(
    loss="categorical_crossentropy", 
    optimizer=keras.optimizers.Adam(learning_rate=0.0001, weight_decay=0.004),
    metrics=['accuracy'],
    weighted_metrics=["accuracy"],
)

offence_model.fit(
    multi_train_x,
    keras.utils.to_categorical(multi_train_y1),
    epochs=100,
    batch_size=32,
    # validation_data=[multi_val_x, keras.utils.to_categorical(multi_val_y1)],
    validation_split=0.2,
    class_weight= multi_class_weights1,
    callbacks= [
        keras.callbacks.ReduceLROnPlateau(
            patience=5,
            factor=0.1,
            monitor='val_loss',
            mode='min'
        ),
        keras.callbacks.EarlyStopping(
            monitor='val_accuracy',
            mode='max',
            patience=10,
            restore_best_weights=True,
        )
    ]
)
offence_model.fit(
    multi_val_x,
    keras.utils.to_categorical(multi_val_y1),
    epochs=100,
    batch_size=32,
    # validation_data=[multi_val_x, keras.utils.to_categorical(multi_val_y1)],
    validation_split=0.2,
    class_weight= multi_class_weights1,
    callbacks= [
        keras.callbacks.ReduceLROnPlateau(
            patience=5,
            factor=0.1,
            monitor='val_loss',
            mode='min'
        ),
        keras.callbacks.EarlyStopping(
            monitor='val_accuracy',
            mode='max',
            patience=10,
            restore_best_weights=True,
        )
    ]
)
offence_model.fit(
    multi_test_x,
    keras.utils.to_categorical(multi_test_y1),
    epochs=100,
    batch_size=32,
    # validation_data=[multi_val_x, keras.utils.to_categorical(multi_val_y1)],
    validation_split=0.2,
    class_weight= multi_class_weights1,
    callbacks= [
        keras.callbacks.ReduceLROnPlateau(
            patience=5,
            factor=0.1,
            monitor='val_loss',
            mode='min'
        ),
        keras.callbacks.EarlyStopping(
            monitor='val_accuracy',
            mode='max',
            patience=10,
            restore_best_weights=True,
        )
    ]
)

Epoch 1/100
232/232 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.3267 - loss: 1.4563 - weighted_accuracy: 0.2342 - val_accuracy: 0.4580 - val_loss: 1.3477 - val_weighted_accuracy: 0.4580 - learning_rate: 1.0000e-04
Epoch 2/100
232/232 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.3537 - loss: 1.4139 - weighted_accuracy: 0.2579 - val_accuracy: 0.2209 - val_loss: 1.3993 - val_weighted_accuracy: 0.2209 - learning_rate: 1.0000e-04
Epoch 3/100
232/232 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.2937 - loss: 1.3644 - weighted_accuracy: 0.2676 - val_accuracy: 0.2635 - val_loss: 1.3797 - val_weighted_accuracy: 0.2635 - learning_rate: 1.0000e-04
Epoch 4/100
232/232 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.3331 - loss: 1.3844 - weighted_accuracy: 0.3270 - val_accuracy: 0.2624 - val_loss: 1.3677 - val_weighted_accuracy: 0.2624 - learning_rate: 1.0000e-04
Epoch 5/100
232/232 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.4226 - loss: 1.3329 - weighted_accuracy: 0.3101 - val_accuracy: 0

In [91]:
chall_x= chall_x.reshape((273, 4, 768))

In [95]:
offence_model.predict(chall_x[1])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step


array([[0.2346264 , 0.2716424 , 0.25659552, 0.2371357 ],
       [0.24175622, 0.2654831 , 0.25031003, 0.24245074],
       [0.24175622, 0.2654831 , 0.25031003, 0.24245074],
       [0.24175622, 0.2654831 , 0.25031003, 0.24245074]], dtype=float32)

In [93]:
#for every 4 clips
#model.predict
#get argmax of every clip
#get max of all clips
#prediction= max

#predictions dictionary
test_predictions= {
    "Actions": {
        
    }
}

for i in np.arange(0, 273):

    #predict
    ac= chall_x[i]
    offence_pred= offence_model.predict(ac)
    type_pred= model.predict(ac)

    y1= []
    for m, el in enumerate(offence_pred):
        y1.append(np.argmax(offence_pred[m]))

    y2= []
    for m, el in enumerate(type_pred):
        y2.append(np.argmax(type_pred[m]))

    majority_y1= np.asarray(y1).max()
    majority_y2= np.asarray(y2).max()
    
    action_number= str(i)
    offence=''
    severity=''

    #divide offence/severity
    if majority_y1==0:
        offence= 'No offence'
    elif majority_y1==1:
        offence= 'Offence'
        severity= '1.0'
    elif majority_y1==2:
        offence= 'Offence'
        severity= '3.0'
    elif majority_y1==3:
        offence= 'Offence'
        severity= '5.0'

    #insert into dictionary
    test_predictions['Actions'][action_number]= {
        'Action class': action_class[majority_y2],
        'Offence': offence,
        'Severity': severity
    }

#save json file
with open("/Users/khuzam/Desktop/fp/SoccerNet/multi_chall_predictions2.json", "w") as outfile: 
    json.dump(test_predictions, outfile)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 167ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━